<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/4_Aprendizaje_no_supervisado/2_Solucion_Taller_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# **Taller: Análisis de Patrones de Consumo Internacional con Apriori**

**IMPORTANTE**: Guarda una copia de este notebook en tu Google Drive o computador.

**Taller en parejas**

**Nombres estudiantes:**

- Carlos Jiménez
- Edgardo Sánchez

**Forma de entrega**

Jupyter Notebook publicado en su cuenta de Github con el nombre “Taller_Apriori_apellidos_estudiantes.ipynb”.

**Plazo de entrega**

Hoy, 16 de octubre, máximo a las 11:59 p.m., debes enviar link del notebook al correo lina.castro6@uexternado.edu.co, de lo contrario, no será tenido en cuenta.

**Instrucciones Generales**

Completa el código en las celdas marcadas con `### TU CÓDIGO AQUÍ ###`. Puedes añadir más celdas si lo requieres.

**Caso de Estudio: Consultoría para el PNUD**

**Contexto:** Una firma multinacional de e-commerce, "Global Retail Inc.", te ha contratado como consultor de datos. La empresa opera en múltiples países y ha notado que sus ventas y la efectividad de sus campañas de marketing varían significativamente entre regiones. Su hipótesis es que los patrones de compra y las asociaciones de productos son diferentes en cada mercado.

**Tu Misión:** Analizar el historial de transacciones de la empresa para descubrir y comparar las reglas de asociación de productos para dos de sus mercados más grandes: el Reino Unido y Alemania. Tu objetivo final es entregar recomendaciones de negocio accionables (ej. estrategias de cross-selling, promociones personalizadas) basadas en los patrones de consumo que descubras en cada país.

**Dataset:** Online Retail Data Set del repositorio de UCI. Encuentra mayor información en: https://archive.ics.uci.edu/dataset/352/online+retail

## Ejercicio 1: Configuración Inicial, Carga y Exploración de Datos

1.1 Importa las librerías necesarias

In [77]:
### TU CÓDIGO AQUÍ ###
import os
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import warnings
warnings.filterwarnings('ignore')

In [78]:
# Configuraciones de visualización
pd.options.display.max_columns = None
pd.options.display.float_format = '{:,.2f}'.format

1.2 Carga el dataset "retail_top30_products.csv" que se encuentra en el repositorio del curso, carpeta "datasets".

In [79]:
### TU CÓDIGO AQUÍ ###
from google.colab import drive
drive.mount('/content/drive')

path="/content/drive/MyDrive/datasets"

# Establecer el directorio
os.chdir(path)

# Leer el archivo CSV
df = pd.read_csv('retail_top30_products.csv')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,"17,850.00",United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,"13,047.00",United Kingdom
2,536368,22960,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,"13,047.00",United Kingdom
3,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,"12,583.00",France
4,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.00,"12,583.00",France
...,...,...,...,...,...,...,...,...
40525,581579,22993,SET OF 4 PANTRY JELLY MOULDS,12,2011-12-09 12:19:00,1.25,"17,581.00",United Kingdom
40526,581583,20725,LUNCH BAG RED RETROSPOT,40,2011-12-09 12:23:00,1.45,"13,777.00",United Kingdom
40527,581585,84879,ASSORTED COLOUR BIRD ORNAMENT,16,2011-12-09 12:31:00,1.69,"15,804.00",United Kingdom
40528,581585,22727,ALARM CLOCK BAKELIKE RED,4,2011-12-09 12:31:00,3.75,"15,804.00",United Kingdom


In [80]:
print("Dimensiones del DataFrame:")
print(df.shape)

Dimensiones del DataFrame:
(40530, 8)


In [81]:
print("\nInformación general del DataFrame:")
df.info()


Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40530 entries, 0 to 40529
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    40530 non-null  object 
 1   StockCode    40530 non-null  object 
 2   Description  40530 non-null  object 
 3   Quantity     40530 non-null  int64  
 4   InvoiceDate  40530 non-null  object 
 5   UnitPrice    40530 non-null  float64
 6   CustomerID   33004 non-null  float64
 7   Country      40530 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 2.5+ MB


1.3 Revisa si hay valores nulos en alguna columna y cuántos son

In [82]:
### TU CÓDIGO AQUÍ ###
df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,7526
Country,0


1.4 Genera las estadísticas descriptivas de las variables numéricas

In [83]:
### TU CÓDIGO AQUÍ ###
df.describe()

,Quantity,UnitPrice,CustomerID
count,"40,530.00","40,530.00","33,004.00"
mean,11.38,4.64,"15,225.76"
std,40.69,57.76,"1,729.80"
min,"-1,930.00",0.00,"12,347.00"
25%,2.00,1.65,"13,767.00"
50%,5.00,2.95,"15,080.00"
75%,10.00,4.95,"16,745.00"
max,"2,880.00","8,142.75","18,283.00"


1.5 Observando las salidas del ejercicio anterior, ¿qué problemas potenciales identificas en las columnas Description, CustomerID y Quantity? ¿Ves algo inusual en los precios (UnitPrice)?

Vemos que no hay problemas en las "Descripción" ya que no hay valores nulos y cada producto tiene su descripción. No obstante, encontramos problemas en el CustomerID ya que identificamos 7,526 valores nulos siendo este un problema ya que sin la identificación del cliente no se pueden asociar las transacciones a los consumidores. En cuanto a la estadística descriptiva, identifican posibles errores y valores atípicos: En la variable Quantity obtenemos un valor negativo, esto puede ser por posibles registros incorrectos o devoluciones; y en el UnitPrice tenemos que hay precios iguales a cero y otros excesivamente altos, viniendo probablemente de errores de digitación o productos gratuitos.



## Ejercicio 2: Limpieza y Preprocesamiento de Datos

Los datos del mundo real rara vez son perfectos. Antes de cualquier análisis, debemos "sanear" nuestro dataset. Completa el código en cada paso según las instrucciones.

2.1 **Manejo de Valores Nulos**: Las transacciones sin un CustomerID no son útiles para nosotros, ya que no podemos agrupar las compras de un cliente específico.

In [84]:
# TAREA: Elimina todas las filas donde 'CustomerID' es nulo.
### TU CÓDIGO AQUÍ ###
df = df.dropna()
df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


In [85]:
# El tipo de dato de CustomerID debe ser entero
### TU CÓDIGO AQUÍ ###
df["CustomerID"] = df["CustomerID"].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33004 entries, 0 to 40529
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    33004 non-null  object 
 1   StockCode    33004 non-null  object 
 2   Description  33004 non-null  object 
 3   Quantity     33004 non-null  int64  
 4   InvoiceDate  33004 non-null  object 
 5   UnitPrice    33004 non-null  float64
 6   CustomerID   33004 non-null  int64  
 7   Country      33004 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 2.3+ MB


2.2 **Limpieza de Descripciones de Productos** Las descripciones pueden tener espacios en blanco al inicio o al final que podrían hacer que un mismo producto se cuente como dos diferentes.

In [86]:
# TAREA: Limpia la columna 'Description' eliminando espacios extra al inicio y al final.
### TU CÓDIGO AQUÍ ###
df['Description'] = df['Description'].str.strip()

2.3 **Filtrado de Transacciones Anómalas**: Las facturas (InvoiceNo) que empiezan con 'C' indican una cancelación. Estas no son compras reales y deben ser eliminadas. Del mismo modo, las cantidades (Quantity) negativas representan devoluciones.

In [87]:
# TAREA: Elimina las filas que correspondan a cancelaciones.
### TU CÓDIGO AQUÍ ###
df_limpio = df[~df['InvoiceNo'].str.contains('C')]


In [99]:
# TAREA: Elimina las filas con cantidades negativas.
### TU CÓDIGO AQUÍ ###
df_limpio = df[df['Quantity'] > 0]

In [100]:
# NO MODIFICAR ESTA CELDA
assert df_limpio['Quantity'].min() > 0, "Error: Todavía hay cantidades negativas."
assert df_limpio['CustomerID'].isnull().sum() == 0, "Error: Aún hay valores nulos en CustomerID."
assert df_limpio[df_limpio['InvoiceNo'].str.contains('C')].empty, "Error: Todavía hay facturas de cancelación."
print("¡La limpieza básica ha sido exitosa!")

¡La limpieza básica ha sido exitosa!


In [101]:
df_limpio

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom
2,536368,22960,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047,United Kingdom
3,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,12583,France
4,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.00,12583,France
...,...,...,...,...,...,...,...,...
40525,581579,22993,SET OF 4 PANTRY JELLY MOULDS,12,2011-12-09 12:19:00,1.25,17581,United Kingdom
40526,581583,20725,LUNCH BAG RED RETROSPOT,40,2011-12-09 12:23:00,1.45,13777,United Kingdom
40527,581585,84879,ASSORTED COLOUR BIRD ORNAMENT,16,2011-12-09 12:31:00,1.69,15804,United Kingdom
40528,581585,22727,ALARM CLOCK BAKELIKE RED,4,2011-12-09 12:31:00,3.75,15804,United Kingdom


## Ejercicio 3: Análisis Comparativo por País

Ahora que los datos están limpios, vamos a segmentarlos y a aplicar el algoritmo Apriori para encontrar los patrones de compra en el Reino Unido y Alemania.

**Preparación de la Cesta de Mercado (Función)**

La siguiente función toma un dataframe, lo agrupa por factura y descripción, y lo transforma en el formato de matriz binaria que necesita el algoritmo Apriori. Estudia esta función, no necesitas modificarla.

In [102]:
# NO MODIFICAR ESTA CELDA
def preparar_cesta(dataframe, pais):
    """Filtra por país y prepara la matriz de transacciones."""

    # Filtrar por el país de interés
    df_pais = dataframe[dataframe['Country'] == pais]

    # Crear la cesta: agrupar productos por factura
    cesta = (df_pais.groupby(['InvoiceNo', 'Description'])['Quantity']
             .sum().unstack().reset_index().fillna(0)
             .set_index('InvoiceNo'))

    # Convertir todas las cantidades positivas a 1 y todo lo demás a 0
    def encode_units(x):
        if x <= 0:
            return 0
        if x >= 1:
            return 1

    cesta_encoded = cesta.applymap(encode_units)
    return cesta_encoded

3.1 Análisis para el Reino Unido (United Kingdom)

In [104]:
# TAREA: Usa la función preparar_cesta para obtener la matriz de transacciones del Reino Unido.
### TU CÓDIGO AQUÍ ###
df_uk = df[df['Country'].eq('United Kingdom')].copy()
basket_uk = preparar_cesta(df_uk, 'United Kingdom')
basket_uk


Description,ALARM CLOCK BAKELIKE RED,ASSORTED COLOUR BIRD ORNAMENT,GREEN REGENCY TEACUP AND SAUCER,HEART OF WICKER SMALL,JAM MAKING SET PRINTED,JAM MAKING SET WITH JARS,JUMBO BAG PINK POLKADOT,JUMBO BAG RED RETROSPOT,JUMBO SHOPPER VINTAGE RED PAISLEY,JUMBO STORAGE BAG SUKI,LUNCH BAG BLACK SKULL.,LUNCH BAG APPLE DESIGN,LUNCH BAG CARS BLUE,LUNCH BAG PINK POLKADOT,LUNCH BAG RED RETROSPOT,LUNCH BAG SPACEBOY DESIGN,LUNCH BAG SUKI DESIGN,NATURAL SLATE HEART CHALKBOARD,PACK OF 72 RETROSPOT CAKE CASES,PAPER CHAIN KIT 50'S CHRISTMAS,PARTY BUNTING,POSTAGE,RECIPE BOX PANTRY YELLOW DESIGN,REGENCY CAKESTAND 3 TIER,ROSES REGENCY TEACUP AND SAUCER,SET OF 3 CAKE TINS PANTRY DESIGN,SET OF 4 PANTRY JELLY MOULDS,SPOTTY BUNTING,WHITE HANGING HEART T-LIGHT HOLDER,WOODEN PICTURE FRAME WHITE FINISH
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
536367,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536371,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
536373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581496,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
581538,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
581579,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [120]:
# TAREA: Aplica el algoritmo apriori para encontrar itemsets con un soporte mínimo de 2%.
### TU CÓDIGO AQUÍ ###
frequent_itemsets = apriori(basket_uk, min_support=0.02, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)
frequent_itemsets

,support,itemsets
27,0.19,(WHITE HANGING HEART T-LIGHT HOLDER)
7,0.14,(JUMBO BAG RED RETROSPOT)
22,0.14,(REGENCY CAKESTAND 3 TIER)
1,0.13,(ASSORTED COLOUR BIRD ORNAMENT)
20,0.13,(PARTY BUNTING)
...,...,...
85,0.02,"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.,..."
94,0.02,"(LUNCH BAG CARS BLUE, LUNCH BAG PINK POLKADOT,..."
92,0.02,"(LUNCH BAG SUKI DESIGN, LUNCH BAG BLACK SKULL..."
35,0.02,"(JAM MAKING SET WITH JARS, SET OF 3 CAKE TINS ..."


In [113]:
# TAREA: Genera las reglas de asociación. Queremos reglas con un Lift mayor a 3.
### TU CÓDIGO AQUÍ ###
rules_uk = association_rules(frequent_itemsets, metric="lift", min_threshold=3)
rules_uk = rules_uk[['antecedents', 'consequents', 'antecedent support', 'consequent support', 'confidence', 'lift']]
rules_uk

,antecedents,consequents,antecedent support,consequent support,confidence,lift
0,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.14,0.08,0.35,4.33
1,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.08,0.14,0.62,4.33
2,(LUNCH BAG RED RETROSPOT),(LUNCH BAG BLACK SKULL.),0.11,0.10,0.43,4.36
3,(LUNCH BAG BLACK SKULL.),(LUNCH BAG RED RETROSPOT),0.10,0.11,0.49,4.36
4,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.06,0.07,0.78,11.54
...,...,...,...,...,...,...
179,"(LUNCH BAG SUKI DESIGN, LUNCH BAG SPACEBOY DES...",(LUNCH BAG BLACK SKULL.),0.03,0.10,0.59,6.01
180,"(LUNCH BAG BLACK SKULL., LUNCH BAG SPACEBOY D...",(LUNCH BAG SUKI DESIGN),0.04,0.08,0.53,6.40
181,(LUNCH BAG SUKI DESIGN),"(LUNCH BAG BLACK SKULL., LUNCH BAG SPACEBOY D...",0.08,0.04,0.25,6.40
182,(LUNCH BAG BLACK SKULL.),"(LUNCH BAG SUKI DESIGN, LUNCH BAG SPACEBOY DES...",0.10,0.03,0.21,6.01


In [116]:
# Ordena las reglas por Lift y Confianza de mayor a menor
### TU CÓDIGO AQUÍ ###
rules_uk.sort_values(['lift', 'confidence'], ascending=[False, False], inplace=True)
rules_uk

,antecedents,consequents,antecedent support,consequent support,confidence,lift
64,"(ROSES REGENCY TEACUP AND SAUCER, REGENCY CAKE...",(GREEN REGENCY TEACUP AND SAUCER),0.04,0.06,0.75,12.37
65,(GREEN REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER, REGENCY CAKE...",0.06,0.04,0.43,12.37
63,"(GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",(ROSES REGENCY TEACUP AND SAUCER),0.03,0.07,0.83,12.24
66,(ROSES REGENCY TEACUP AND SAUCER),"(GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",0.07,0.03,0.39,12.24
5,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.07,0.06,0.70,11.54
...,...,...,...,...,...,...
113,(JUMBO BAG PINK POLKADOT),(LUNCH BAG PINK POLKADOT),0.08,0.08,0.28,3.37
90,(SET OF 3 CAKE TINS PANTRY DESIGN),(RECIPE BOX PANTRY YELLOW DESIGN),0.10,0.07,0.24,3.25
91,(RECIPE BOX PANTRY YELLOW DESIGN),(SET OF 3 CAKE TINS PANTRY DESIGN),0.07,0.10,0.33,3.25
32,(PARTY BUNTING),(SPOTTY BUNTING),0.13,0.09,0.28,3.08


3.3 Observa las 3 reglas con el Lift más alto para el Reino Unido (1, 3 y 5). **Interprétalas:** ¿Qué te dicen estas asociaciones? ¿Qué tipo de productos son?

Las tres reglas con mayor valor de Lift nos muestra una relación entre los productos de la línea "Regency", sinedo estos: “Roses Regency Teacup and Saucer”, “Green Regency Teacup and Saucer” y “Regency Cake Stand”. Esto significa que los consumidores suelen comprar productos complementarios a esta marca tras haber adquirido su primera pieza de este tipo de vajilla. El hecho de que el Lift sea superior a 12 indica que estas combinaciones se presentan más de doce veces con mayor frecuencia de lo esperado si las compras fueran independientes, lo que refleja una asociación extremadamente fuerte en el ámbito británico.

3.4 Interpreta el Soporte para el antecedente y el consecuente, la Confianza y el Lift

El Soporte representa la proporción de transacciones en las que aparece la combinación de productos, por ejemplo, un valor de 0.04 significa que el 4 % de las compras incluyen ambos artículos. La Confianza, en cambio, mide la probabilidad de que un cliente que compre un primer producto adquiera también el seguiente; una confianza del 0.75 indica que el 75 % de los clientes que compran una taza “Roses Regency” también adquieren la “Green Regency”. Finalmente, el Lift compara la frecuencia observada con la esperada si los productos fueran independientes; un valor de Lift mayor que 1 señala una relación positiva, y en este caso, valores superiores a 12 revelan una conexión muy fuerte y no aleatoria. En conjunto, estos resultados demuestran patrones de compra hacia estas colecciones.

3.5 **Recomendación de Negocio:** Basado en estas reglas, ¿qué promoción o estrategia de venta específica podrías sugerir para el mercado del Reino Unido?

Dado que las reglas más relevantes provienen de productos de una misma línea, se recomienda aprovechar esta complementariedad mediante estrategias de venta cruzada y promoción conjunta. Una opción es crear combos o packs “Regency Tea Set”, ofreciendo descuentos al comprar varias piezas de la colección (por ejemplo, taza, plato y soporte para pastel). También sería conveniente diseñar empaques de regalo personalizados para atraer a compradores que buscan obsequios elegantes y coherentes visualmente, especialmente en temporadas de alta demanda como Navidad o el Día de la Madre. Además, el uso de recomendaciones automáticas en plataformas en línea puede aumentar las ventas al sugerir piezas complementarias tras una compra inicial. Estas estrategias no solo refuerzan el atractivo de la marca, sino que también elevan el valor promedio de cada transacción y fortalecen la fidelización del cliente.

3.6 Análisis para Alemania (Germany)

In [117]:
# TAREA: Usa la función preparar_cesta para obtener la matriz de transacciones de Alemania.
### TU CÓDIGO AQUÍ ###
df_germany = df[df['Country'].eq('Germany')].copy()
basket_germany = preparar_cesta(df_germany, 'Germany')
basket_germany

Description,ALARM CLOCK BAKELIKE RED,ASSORTED COLOUR BIRD ORNAMENT,GREEN REGENCY TEACUP AND SAUCER,HEART OF WICKER SMALL,JAM MAKING SET PRINTED,JAM MAKING SET WITH JARS,JUMBO BAG PINK POLKADOT,JUMBO BAG RED RETROSPOT,JUMBO SHOPPER VINTAGE RED PAISLEY,JUMBO STORAGE BAG SUKI,LUNCH BAG BLACK SKULL.,LUNCH BAG APPLE DESIGN,LUNCH BAG CARS BLUE,LUNCH BAG PINK POLKADOT,LUNCH BAG RED RETROSPOT,LUNCH BAG SPACEBOY DESIGN,LUNCH BAG SUKI DESIGN,NATURAL SLATE HEART CHALKBOARD,PACK OF 72 RETROSPOT CAKE CASES,PAPER CHAIN KIT 50'S CHRISTMAS,PARTY BUNTING,POSTAGE,RECIPE BOX PANTRY YELLOW DESIGN,REGENCY CAKESTAND 3 TIER,ROSES REGENCY TEACUP AND SAUCER,SET OF 3 CAKE TINS PANTRY DESIGN,SET OF 4 PANTRY JELLY MOULDS,SPOTTY BUNTING,WHITE HANGING HEART T-LIGHT HOLDER,WOODEN PICTURE FRAME WHITE FINISH
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
536840,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581266,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
581494,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
581570,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [119]:
# TAREA: Aplica el algoritmo apriori con un soporte mínimo del 2%.
### TU CÓDIGO AQUÍ ###
frequent_itemsets = apriori(basket_germany, min_support=0.02, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)
frequent_itemsets

,support,itemsets
15,0.90,(POSTAGE)
17,0.15,(REGENCY CAKESTAND 3 TIER)
38,0.13,"(POSTAGE, REGENCY CAKESTAND 3 TIER)"
2,0.10,(JAM MAKING SET PRINTED)
14,0.09,(PACK OF 72 RETROSPOT CAKE CASES)
5,0.09,(JUMBO BAG RED RETROSPOT)
24,0.08,"(JAM MAKING SET PRINTED, POSTAGE)"
29,0.07,"(JUMBO BAG RED RETROSPOT, POSTAGE)"
9,0.07,(LUNCH BAG APPLE DESIGN)
18,0.07,(SET OF 3 CAKE TINS PANTRY DESIGN)


In [121]:
# TAREA: Genera las reglas de asociación con un Lift mayor a 3.
### TU CÓDIGO AQUÍ ###
rules_germany = association_rules(frequent_itemsets, metric="lift", min_threshold=3)
rules_germany = rules_germany[['antecedents', 'consequents', 'antecedent support', 'consequent support', 'confidence', 'lift']]
rules_germany

,antecedents,consequents,antecedent support,consequent support,confidence,lift
0,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.14,0.08,0.35,4.33
1,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.08,0.14,0.62,4.33
2,(LUNCH BAG RED RETROSPOT),(LUNCH BAG BLACK SKULL.),0.11,0.10,0.43,4.36
3,(LUNCH BAG BLACK SKULL.),(LUNCH BAG RED RETROSPOT),0.10,0.11,0.49,4.36
4,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.06,0.07,0.78,11.54
...,...,...,...,...,...,...
179,"(LUNCH BAG SUKI DESIGN, LUNCH BAG SPACEBOY DES...",(LUNCH BAG BLACK SKULL.),0.03,0.10,0.59,6.01
180,"(LUNCH BAG BLACK SKULL., LUNCH BAG SPACEBOY D...",(LUNCH BAG SUKI DESIGN),0.04,0.08,0.53,6.40
181,(LUNCH BAG SUKI DESIGN),"(LUNCH BAG BLACK SKULL., LUNCH BAG SPACEBOY D...",0.08,0.04,0.25,6.40
182,(LUNCH BAG BLACK SKULL.),"(LUNCH BAG SUKI DESIGN, LUNCH BAG SPACEBOY DES...",0.10,0.03,0.21,6.01


In [122]:
# Ordena las reglas por Lift y Confianza de mayor a menor
### TU CÓDIGO AQUÍ ###
rules_germany.sort_values(['lift', 'confidence'], ascending=[False, False], inplace=True)
rules_germany

,antecedents,consequents,antecedent support,consequent support,confidence,lift
64,"(ROSES REGENCY TEACUP AND SAUCER, REGENCY CAKE...",(GREEN REGENCY TEACUP AND SAUCER),0.04,0.06,0.75,12.37
65,(GREEN REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER, REGENCY CAKE...",0.06,0.04,0.43,12.37
63,"(GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",(ROSES REGENCY TEACUP AND SAUCER),0.03,0.07,0.83,12.24
66,(ROSES REGENCY TEACUP AND SAUCER),"(GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",0.07,0.03,0.39,12.24
5,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.07,0.06,0.70,11.54
...,...,...,...,...,...,...
113,(JUMBO BAG PINK POLKADOT),(LUNCH BAG PINK POLKADOT),0.08,0.08,0.28,3.37
90,(SET OF 3 CAKE TINS PANTRY DESIGN),(RECIPE BOX PANTRY YELLOW DESIGN),0.10,0.07,0.24,3.25
91,(RECIPE BOX PANTRY YELLOW DESIGN),(SET OF 3 CAKE TINS PANTRY DESIGN),0.07,0.10,0.33,3.25
32,(PARTY BUNTING),(SPOTTY BUNTING),0.13,0.09,0.28,3.08


3.7 Observa las 3 reglas con el Lift más alto para Alemania (1, 3 y 5). **Interprétalas:** ¿Qué patrones de consumo específicos del mercado alemán revelan estas reglas? ¿Son diferentes a los del Reino Unido?



Las tres reglas con mayor valor de Lift en el mercado alemán evidencian una relación entre productos de decoración doméstica y artículos para celebraciones, como “Jumbo Bag Pink Polkadot”, “Recipe Box Pantry Yellow Design” o “Spotty Bunting”. Estos artículos reflejan un patrón de consumo orientado hacia la organización del hogar y la estética del espacio cotidiano, así como hacia la planificación de eventos o reuniones familiares. A diferencia del Reino Unido, donde las asociaciones giraban en torno a vajillas y artículos complementarios de una misma colección, en Alemania se observa una preferencia por productos prácticos y funcionales, con un componente visual o decorativo más marcado pero menos dependiente de la combinación dentro de un set. Esto sugiere que los consumidores alemanes buscan productos individuales que mantengan coherencia estética, pero que pueden adquirirse por separado sin necesidad de completar una línea.

3.8 Interpreta el Soporte para el antecedente y el consecuente, la Confianza y el Lift

En las reglas de asociación del mercado alemán, el Soporte se sitúa entre 0.07 y 0.13, lo que indica que entre el 7 % y el 13 % de las transacciones incluyen esas combinaciones de productos, un nivel moderado pero significativo. La Confianza varía entre 0.24 y 0.40, mostrando que, si bien las asociaciones no son tan fuertes como las del Reino Unido, sí reflejan hábitos de compra consistentes en artículos relacionados con decoración, empaques y utensilios para el hogar. El Lift, que oscila entre 3.08 y 3.37, es menor al observado en Reino Unido (donde superaba 11), pero sigue indicando relaciones positivas: estos productos se compran juntos más de tres veces de lo que se esperaría por azar. En conjunto, estas métricas reflejan un comportamiento de consumo más diverso y funcional, donde las combinaciones responden a la complementariedad práctica más que a la pertenencia a una colección cerrada.

3.9 **Recomendación de Negocio:** ¿Qué campaña de marketing (diferente a la del Reino Unido) podrías diseñar para los clientes alemanes?

A partir de estos resultados, se recomienda diseñar para Alemania una campaña de marketing enfocada en la funcionalidad y la ambientación del hogar, diferente a la estrategia estética y coleccionista aplicada en el Reino Unido. Una idea concreta sería lanzar una campaña temática de “Decoración práctica para el día a día”, promoviendo combos o descuentos por la compra de artículos de uso complementario —por ejemplo, bolsas decorativas, cajas organizadoras y adornos de mesa— bajo el mensaje de crear espacios agradables con poco esfuerzo. Además, se podrían utilizar estrategias de email marketing segmentado, recomendando productos relacionados con fiestas o celebraciones, dado que varios artículos están vinculados a ese tipo de ocasión. En general, las estrategias para Alemania deberían resaltar la utilidad, el diseño simple y la relación calidad-precio, elementos clave en el comportamiento de compra de este mercado.